# Cloud Workshop Azure Databricks
## 08. Cross Validation
<img src="https://raw.githubusercontent.com/retkowsky/images/master/AzureDatabricksLogo.jpg"><br>
V1.3 12/06/2019

## Using Cross Validation

In this exercise, you will use cross-validation to optimize parameters for a regression model.

### Prepare the Data

First, import the libraries you will need and prepare the training and test data:

In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler, StringIndexer, MinMaxScaler
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

# Load the source data
# File location and type
file_location = "/FileStore/tables/flights.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
data = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

data = data.select("DayofMonth", "DayOfWeek", "Carrier", "OriginAirportID", "DestAirportID", "DepDelay", col("ArrDelay").alias("label"))

# Split the data
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1]

### Define the Pipeline
Now define a pipeline that creates a feature vector and trains a regression model

In [5]:
monthdayIndexer = StringIndexer(inputCol="DayofMonth", outputCol="DayofMonthIdx")
weekdayIndexer = StringIndexer(inputCol="DayOfWeek", outputCol="DayOfWeekIdx")
carrierIndexer = StringIndexer(inputCol="Carrier", outputCol="CarrierIdx")
originIndexer = StringIndexer(inputCol="OriginAirportID", outputCol="OriginAirportIdx")
destIndexer = StringIndexer(inputCol="DestAirportID", outputCol="DestAirportIdx")
numVect = VectorAssembler(inputCols = ["DepDelay"], outputCol="numFeatures")
minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), outputCol="normNums")

featVect = VectorAssembler(inputCols=["DayofMonthIdx", "DayOfWeekIdx", "CarrierIdx", "OriginAirportIdx", "DestAirportIdx", "normNums"], outputCol="features")

lr = LinearRegression(labelCol="label", featuresCol="features")

pipeline = Pipeline(stages=[monthdayIndexer, weekdayIndexer, carrierIndexer, originIndexer, destIndexer, numVect, minMax, featVect, lr])

### Tune Parameters
You can tune parameters to find the best model for your data. <br>
To do this you can use the  **CrossValidator** class to evaluate each combination of parameters defined in a **ParameterGrid** against multiple *folds* of the data split into training and validation datasets, in order to find the best performing parameters. <br> 
> Note that this can take a long time to run because every parameter combination is tried multiple times.

In [7]:
paramGrid = ParamGridBuilder().addGrid(lr.regParam, [0.3, 0.01]).addGrid(lr.maxIter, [10, 5]).build()
cv = CrossValidator(estimator=pipeline, evaluator=RegressionEvaluator(), estimatorParamMaps=paramGrid, numFolds=2)

model = cv.fit(train)

### Test the Model
Now you're ready to apply the model to the test data.

In [9]:
prediction = model.transform(test)
predicted = prediction.select("features", "prediction", "label")

predicted.show()

+--------------------+--------------------+-----+
 features| prediction|label|
+--------------------+--------------------+-----+
[25.0,2.0,10.0,47...| 15.002827149090841| 7|
[25.0,2.0,10.0,38...| 60.983791015403014| 56|
[25.0,2.0,10.0,38...|-0.45091450420016344| 62|
[25.0,2.0,10.0,38...| 53.37326770938071| 44|
[25.0,2.0,10.0,23...| 57.78912431935339| 50|
[25.0,2.0,10.0,14...| 2.8562040151252575| 37|
[25.0,2.0,10.0,12...| 44.47701817935054| 57|
[25.0,2.0,10.0,12...| 1.9048181825805415| 67|
[25.0,2.0,10.0,12...| 2.2117975135029226| -28|
[25.0,2.0,10.0,0....| 42.78578478238566| 32|
[25.0,2.0,10.0,67...| 4.048038057414217| -20|
[25.0,2.0,2.0,34....| 16.699079335319766| 6|
[25.0,2.0,2.0,27....|-0.01921402862395638| 26|
[25.0,2.0,2.0,11....| 17.224286117936273| 35|
[25.0,2.0,2.0,11....| 38.916167469194434| 44|
[25.0,2.0,2.0,3.0...| 5.042592472589476| -8|
[25.0,2.0,2.0,3.0...| 41.999374626024036| 28|
[25.0,2.0,2.0,3.0...| 0.3047788085066685| -10|
[25.0,2.0,2.0,3.0...| 0.3047788085066685| -10|
[25.0,2.0,2.0,3.0...|-0.33136713671260765| -16|
+--------------------+--------------------+-----+
only showing top 20 rows

### Examine the Predicted and Actual Values
You can plot the predicted values against the actual values to see how accurately the model has predicted. In a perfect model, the resulting scatter plot should form a perfect diagonal line with each predicted value being identical to the actual value - in practice, some variance is to be expected.
Run the cells below to create a temporary table from the **predicted** DataFrame and then retrieve the predicted and actual label values using SQL. You can then display the results as a scatter plot, specifying **-** as the function to show the unaggregated values.

In [11]:
predicted.createOrReplaceTempView("regressionPredictions")

In [12]:
%sql
SELECT label, prediction FROM regressionPredictions

label,prediction
7,15.002827149090841
56,60.983791015403014
62,-0.45091450420016344
44,53.37326770938071
50,57.78912431935339
37,2.8562040151252575
57,44.47701817935054
67,1.9048181825805415
-28,2.2117975135029226
32,42.78578478238566


### Retrieve the Root Mean Square Error (RMSE)
There are a number of metrics used to measure the variance between predicted and actual values. Of these, the root mean square error (RMSE) is a commonly used value that is measured in the same units as the prediced and actual values - so in this case, the RMSE indicates the average number of minutes between predicted and actual flight delay values. You can use the **RegressionEvaluator** class to retrieve the RMSE.

In [14]:
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(prediction)
print ("Root Mean Square Error (RMSE):", rmse)

Root Mean Square Error (RMSE): 17.240157301984283

> Fin